In [1]:
import pandas as pd
import os
# daily_agg_county_datapath = "/data/BHI_G5/daily_agg_county/county_daily_aggregated_timeseries.csv"
daily_agg_county_datapath = "county_daily_aggregated_timeseries.csv"
agg_df = pd.read_csv(daily_agg_county_datapath,index_col=0)

In [2]:
agg_df

,actuals.cases,actuals.deaths,metrics.deathDensity,metrics.caseDensity,metrics.testPositivityRatio,cdcTransmissionLevel,riskLevels.caseDensity,riskLevels.overall,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
date,,,,,,,,,,
2020-04-05,331769.0,9285.0,0.031711,9417.1,0.106803,0.448738,0.096146,0.195474,0.018569,0.058003
2020-04-06,362217.0,10662.0,0.038168,10161.4,0.105619,0.451557,0.100693,0.200321,0.018569,0.058003
2020-04-07,393641.0,12748.0,0.045717,10931.2,0.106687,0.453574,0.106921,0.206631,0.018569,0.058003
2020-04-08,425086.0,14655.0,0.050901,11404.1,0.106887,0.461696,0.109792,0.208432,0.018569,0.058003
2020-04-09,459307.0,16262.0,0.056929,11825.5,0.106429,0.463041,0.111778,0.209332,0.018569,0.058003
...,...,...,...,...,...,...,...,...,...,...
2022-01-05,56845503.0,824756.0,9.721594,351625.4,0.274392,0.992086,0.837409,0.840451,0.492552,0.555811
2022-01-06,57543129.0,826494.0,9.745629,377290.1,0.282382,0.992189,0.858297,0.860656,0.493028,0.556442
2022-01-07,58703383.0,828892.0,9.768816,408058.5,0.288566,0.993224,0.879847,0.881047,0.493438,0.557038


In [3]:
agg_df = (agg_df - agg_df.mean())/agg_df.std()
agg_df.head()

,actuals.cases,actuals.deaths,metrics.deathDensity,metrics.caseDensity,metrics.testPositivityRatio,cdcTransmissionLevel,riskLevels.caseDensity,riskLevels.overall,metrics.vaccinationsCompletedRatio,metrics.vaccinationsInitiatedRatio
date,,,,,,,,,,
2020-04-05,-1.415663,-1.731067,-1.446085,-1.119274,0.177391,-1.280439,-1.604119,-1.291902,-0.902101,-0.934697
2020-04-06,-1.413819,-1.725364,-1.443964,-1.108633,0.149367,-1.267468,-1.579321,-1.263114,-0.902101,-0.934697
2020-04-07,-1.411916,-1.716725,-1.441485,-1.097628,0.174666,-1.258186,-1.545354,-1.225631,-0.902101,-0.934697
2020-04-08,-1.410012,-1.708828,-1.439783,-1.090867,0.179394,-1.220820,-1.529697,-1.214940,-0.902101,-0.934697
2020-04-09,-1.407940,-1.702173,-1.437803,-1.084843,0.168560,-1.214632,-1.518864,-1.209594,-0.902101,-0.934697


In [ ]:
def create_X_Y(ts, lag=1, n_ahead=1, target_index=0) -> tuple:
    """
    A method to create X and Y matrix from a time series array for the training of 
    deep learning models 
    """
    # Extracting the number of features that are passed from the array 
    n_features = ts.shape[1]
    
    # Creating placeholder lists
    X, Y = [], []

    if len(ts) - lag <= 0:
        X.append(ts)
    else:
        for i in range(len(ts) - lag - n_ahead):
            Y.append(ts[(i + lag):(i + lag + n_ahead), target_index])
            X.append(ts[i:(i + lag)])

    X, Y = np.array(X), np.array(Y)

    # Reshaping the X array to an RNN input shape 
    X = np.reshape(X, (X.shape[0], lag, n_features))

    return X, Y

In [43]:
cases_mean = agg_df.mean()['metrics.caseDensity']
deaths_mean = agg_df.mean()['metrics.deathDensity']
cases_std = agg_df.std()['metrics.caseDensity']
deaths_std = agg_df.std()['metrics.deathDensity']
cases_mean, deaths_mean,cases_std,deaths_std

import json
scaling_info={
    "cases_mean": cases_mean,
    "deaths_mean":deaths_mean,
    "cases_std": cases_std,
    "deaths_std":deaths_std
}
with open('daily_data_scaling_inforamtion.json', 'w+') as outfile:
    json.dump(scaling_info, outfile)

## Timeseries to Supervised

In [4]:
def series_to_supervised(df, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(df) is list else df.shape[1]
    # n_vars_output = 2
    # df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('%s(t-%d)' % (df.columns[j], i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i)) #Output Variables
        if i == 0:
            names += [('%s(t)' % (df.columns[j])) for j in range(n_vars)]
        else:
            names += [('%s(t+%d)' % (df.columns[j], i)) for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [5]:
def create_supervised_X_Y(dataframe,no_output_steps,no_input_steps ):
    #Frame as supervised learning and drop all time t columns except
    no_features = dataframe.shape[1]
    # no_output_steps = 7
    # no_input_steps = 14
    slice_index = int(no_features*no_output_steps)
    
    reframed_dataframe = series_to_supervised(dataframe, no_input_steps, no_output_steps)
    output_features = []
    for feature in reframed_dataframe.columns.tolist():
        if ("metrics.caseDensity" in feature or "metrics.deathDensity" in feature) and ("-" not in feature):
            output_features.append(feature)
    
    X_data = reframed_dataframe.iloc[:,:-slice_index]
    Y_data = reframed_dataframe.iloc[:,-slice_index:][output_features]

    return X_data,Y_data

In [50]:
def mu_sigma_feature_engineering(X_data):
    
    final_features = []
    for feature in X_data.columns.tolist():
        final_features.append(feature.split("(")[0])
    final_features = list(set(final_features))   
    features_dict = {feature: [] for feature in final_features}

    for feature in X_data.columns.tolist():
        features_dict[feature.split("(")[0]].append(feature)

    new_X_data = pd.DataFrame()
    for feature, feature_list in features_dict.items():
        # print(feature_list)
        df = X_data[feature_list]

        stddev = str(feature) +"(14day_stddev)"
        mean =  str(feature) +"(14day_mean)"
        # engineered_df = pd.DataFrame()
        new_X_data[mean] = df.mean(axis=1)
        new_X_data[stddev] = df.std(axis=1)

    return new_X_data

In [55]:
def train_val_test_split(X_data,Y_data, output_path, test_slice):

    train_val_X = X_data.iloc[:-test_slice]
    train_val_Y = Y_data.iloc[:-test_slice]
    
    X_test = X_data.iloc[-test_slice:]
    Y_test = Y_data.iloc[-test_slice:]
    
    size=int(len(train_val_X)*0.80)
    X_train, X_val = train_val_X.iloc[0:size], train_val_X.iloc[size:]
    Y_train, Y_val = train_val_Y.iloc[0:size], train_val_Y.iloc[size:]
    
    full_output_path = output_path + "/"
#     full_output_path = "/data/BHI_G5/daily_agg_county/" + output_path + "/"
    
    X_train.to_csv(full_output_path+"x_train.csv",index=True)
    Y_train.to_csv(full_output_path+"y_train.csv",index=True)
    
    X_val.to_csv(full_output_path+"x_val.csv",index=True)
    Y_val.to_csv(full_output_path+"y_val.csv",index=True)
    
    X_test.to_csv(full_output_path+"x_test.csv",index=True)
    Y_test.to_csv(full_output_path+"y_test.csv",index=True)
    
    return 

In [56]:
def create_folder(dir):
    import os
    import shutil
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [57]:
def create_dataframes(agg_df, output_steps, input_steps, output_folder, test_slice):
    X_data,Y_data = create_supervised_X_Y(agg_df,no_output_steps=output_steps,no_input_steps=input_steps)
    print(X_data.shape)
    print(Y_data.shape)
    
    mu_sigma_X_data = mu_sigma_feature_engineering(X_data)
    print(mu_sigma_X_data.shape)
    
    output_path = output_folder+ "/" + str(input_steps)+"DayWin_" + str(output_steps) + "DayPred"
    create_folder(output_path)
    train_val_test_split(X_data,Y_data,output_path,test_slice)
    
    mu_sigma_output_path = output_folder+ "/" + str(input_steps)+"DayWin_MuSigma_" + str(output_steps) + "DayPred"
    create_folder(mu_sigma_output_path)
    train_val_test_split(mu_sigma_X_data,Y_data,mu_sigma_output_path, test_slice)
    
    print(output_path,mu_sigma_output_path)
    return

In [58]:
create_dataframes(agg_df,output_steps=7,input_steps=14, output_folder = "Delta_Omicron_Pred", test_slice=159)
create_dataframes(agg_df,output_steps=7,input_steps=21, output_folder = "Delta_Omicron_Pred", test_slice=159)

create_dataframes(agg_df,output_steps=7,input_steps=14, output_folder = "Dec28_2021_Pred", test_slice=7)
create_dataframes(agg_df,output_steps=7,input_steps=21, output_folder = "Dec28_2021_Pred", test_slice=7)

create_dataframes(agg_df,output_steps=7,input_steps=14, output_folder = "Jan3_2022_Pred", test_slice=1)
create_dataframes(agg_df,output_steps=7,input_steps=21, output_folder = "Jan3_2022_Pred", test_slice=1)

(625, 140)
(625, 14)
(625, 20)
Delta_Omicron_Pred/14DayWin_7DayPred Delta_Omicron_Pred/14DayWin_MuSigma_7DayPred
(618, 210)
(618, 14)
(618, 20)
Delta_Omicron_Pred/21DayWin_7DayPred Delta_Omicron_Pred/21DayWin_MuSigma_7DayPred
(625, 140)
(625, 14)
(625, 20)
Dec28_2021_Pred/14DayWin_7DayPred Dec28_2021_Pred/14DayWin_MuSigma_7DayPred
(618, 210)
(618, 14)
(618, 20)
Dec28_2021_Pred/21DayWin_7DayPred Dec28_2021_Pred/21DayWin_MuSigma_7DayPred
(625, 140)
(625, 14)
(625, 20)
Jan3_2022_Pred/14DayWin_7DayPred Jan3_2022_Pred/14DayWin_MuSigma_7DayPred
(618, 210)
(618, 14)
(618, 20)
Jan3_2022_Pred/21DayWin_7DayPred Jan3_2022_Pred/21DayWin_MuSigma_7DayPred
